# This notebook based on kaggle notebook:https://www.kaggle.com/gspmoreira/recommender-systems-in-python-101 Please refer to the base notebook for more details 

# import dataset

In [1]:
import pandas as pd
article_df=pd.read_csv('../archive/shared_articles.csv')
interactions_df=pd.read_csv('../archive/users_interactions.csv')
    

In [2]:
article_df.head(3)

,timestamp,eventType,contentId,authorPersonId,authorSessionId,authorUserAgent,authorRegion,authorCountry,contentType,url,title,text,lang
0,1459192779,CONTENT REMOVED,-6451309518266745024,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,http://www.nytimes.com/2016/03/28/business/dea...,"Ethereum, a Virtual Currency, Enables Transact...",All of this work is still very early. The firs...,en
1,1459193988,CONTENT SHARED,-4110354420726924665,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,http://www.nytimes.com/2016/03/28/business/dea...,"Ethereum, a Virtual Currency, Enables Transact...",All of this work is still very early. The firs...,en
2,1459194146,CONTENT SHARED,-7292285110016212249,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,http://cointelegraph.com/news/bitcoin-future-w...,Bitcoin Future: When GBPcoin of Branson Wins O...,The alarm clock wakes me at 8:00 with stream o...,en


In [3]:
article_df['eventType'].unique()

array(['CONTENT REMOVED', 'CONTENT SHARED'], dtype=object)

In [4]:
# get only eventtype='CONTENT SHARED'

article_df=article_df[article_df['eventType']=='CONTENT SHARED']

In [5]:
article_df.shape

(3047, 13)

In [6]:
interactions_df['eventType'].unique()

array(['VIEW', 'FOLLOW', 'BOOKMARK', 'LIKE', 'COMMENT CREATED'],
      dtype=object)

# Feature engineering

## filling the users-items matrix

In [7]:
event_type_strength = {
   'VIEW': 1.0,
   'LIKE': 2.0, 
   'BOOKMARK': 2.5, 
   'FOLLOW': 3.0,
   'COMMENT CREATED': 4.0,  
}


interactions_df['eventStrength']=interactions_df['eventType'].apply(lambda x: event_type_strength[x])

### Users Can interact with aticles multiple times 

In [20]:
interactions_df.groupby(['personId', 'contentId']).size()

personId              contentId           
-9223121837663643404  -8949113594875411859    1
                      -8377626164558006982    1
                      -8208801367848627943    1
                      -8187220755213888616    1
                      -7423191370472335463    8
                                             ..
 9210530975708218054   8477804012624580461    4
                       8526042588044002101    1
                       8856169137131817223    1
                       8869347744613364434    1
                       9209886322932807692    1
Length: 40710, dtype: int64

In [9]:
interactions_df[(interactions_df['personId']==9210530975708218054) & (interactions_df['contentId']==8477804012624580461)].sort_values(by=['timestamp'])

,timestamp,eventType,contentId,personId,sessionId,userAgent,userRegion,userCountry,eventStrength
68300,1486577685,VIEW,8477804012624580461,9210530975708218054,3436065331471531836,Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/53...,SP,BR,1.0
68290,1486577725,BOOKMARK,8477804012624580461,9210530975708218054,3436065331471531836,Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/53...,SP,BR,2.5
68291,1486577726,LIKE,8477804012624580461,9210530975708218054,3436065331471531836,Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/53...,SP,BR,2.0
68297,1486577729,FOLLOW,8477804012624580461,9210530975708218054,3436065331471531836,Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/53...,SP,BR,3.0


In [10]:
interactions_df[(interactions_df['personId']==-9223121837663643404) & (interactions_df['contentId']==-7423191370472335463)].sort_values(by=['timestamp'])

,timestamp,eventType,contentId,personId,sessionId,userAgent,userRegion,userCountry,eventStrength
60988,1478687166,VIEW,-7423191370472335463,-9223121837663643404,2411828781515959462,Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/53...,SP,BR,1.0
64003,1478705793,VIEW,-7423191370472335463,-9223121837663643404,2411828781515959462,Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/53...,SP,BR,1.0
61256,1478795892,VIEW,-7423191370472335463,-9223121837663643404,-4096620279830729916,Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/53...,SP,BR,1.0
63155,1478860184,VIEW,-7423191370472335463,-9223121837663643404,844431470588928965,Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/53...,SP,BR,1.0
63183,1478860897,VIEW,-7423191370472335463,-9223121837663643404,844431470588928965,Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/53...,SP,BR,1.0
63180,1478862151,VIEW,-7423191370472335463,-9223121837663643404,844431470588928965,Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/53...,SP,BR,1.0
65167,1479290925,VIEW,-7423191370472335463,-9223121837663643404,-8865295409778928525,Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/53...,SP,BR,1.0
63834,1479376578,VIEW,-7423191370472335463,-9223121837663643404,-6084152871682607649,Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/53...,SP,BR,1.0


In [11]:
users_interactions_count_df = interactions_df.groupby(['personId', 'contentId']).size().groupby('personId').size()
users_with_enough_interactions_df = users_interactions_count_df[users_interactions_count_df >= 5].reset_index()[['personId']]

print('# users with at least 5 interactions: %d' % len(users_with_enough_interactions_df))

# users with at least 5 interactions: 1140


### Get total interactions from users with at least 5 articles 

In [21]:
interactions_from_selected_users_df = interactions_df.merge(users_with_enough_interactions_df,how='right',left_on='personId',right_on='personId')

print('# of interactions from users with at least 5 interactions: %d' % len(interactions_from_selected_users_df))

# of interactions from users with at least 5 interactions: 69868


In [22]:
interactions_from_selected_users_df.groupby(['eventType','eventStrength']).size()

eventType        eventStrength
BOOKMARK         2.5               2420
COMMENT CREATED  4.0               1579
FOLLOW           3.0               1379
LIKE             2.0               5625
VIEW             1.0              58865
dtype: int64

### Apply log transformation to smooth the distribution

In [23]:
import math
def smooth_user_preference(x):
    return math.log(1+x, 2)

interactions_full_df=interactions_from_selected_users_df \
                    .groupby(['personId', 'contentId'])['eventStrength'].sum()\
                    .apply(smooth_user_preference).reset_index()

print('# of unique user/item interactions: %d' % len(interactions_full_df))
interactions_full_df.head(10)

# of unique user/item interactions: 39106


,personId,contentId,eventStrength
0,-9223121837663643404,-8949113594875411859,1.000000
1,-9223121837663643404,-8377626164558006982,1.000000
2,-9223121837663643404,-8208801367848627943,1.000000
3,-9223121837663643404,-8187220755213888616,1.000000
4,-9223121837663643404,-7423191370472335463,3.169925
5,-9223121837663643404,-7331393944609614247,1.000000
6,-9223121837663643404,-6872546942144599345,1.000000
7,-9223121837663643404,-6728844082024523434,1.000000
8,-9223121837663643404,-6590819806697898649,1.000000
9,-9223121837663643404,-6558712014192834002,1.584963


# items profiles 

Use TFIDF and cosine similarity for items feature engineerings 

In [27]:
from nltk.corpus import stopwords


In [28]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /Users/amr/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [17]:
from sklearn.feature_extraction.text import TfidfVectorizer

stopwords_list = stopwords.words('english') + stopwords.words('portuguese')

vectorizer = TfidfVectorizer(analyzer='word',
                     ngram_range=(1, 2),
                     min_df=0.003,
                     max_df=0.5,
                     max_features=5000,
                     stop_words=stopwords_list)

tfidf_matrix = vectorizer.fit_transform(article_df['title'] + "" + article_df['text'])

In [18]:
item_ids = article_df['contentId'].tolist()

In [19]:
tfidf_matrix.shape

(3047, 5000)

# Build train and test sets

In [ ]:
from sklearn.model_selection import train_test_split

interactions_train_df, interactions_test_df = train_test_split(interactions_full_df,
                                   stratify=interactions_full_df['personId'], 
                                   test_size=0.20,
                                   random_state=42)

In [ ]:
interactions_train_df[interactions_train_df['personId']==3609194402293569455].groupby('personId').size()

In [ ]:
interactions_test_df[interactions_test_df['personId']==3609194402293569455].groupby('personId').size()

# Build user profiles 

In [31]:
import scipy
import numpy as np
import sklearn

def get_item_profile(item_id):
    idx = item_ids.index(item_id)
    item_profile = tfidf_matrix[idx:idx+1]
    return item_profile

def get_item_profiles(ids):
    item_profiles_list = [get_item_profile(x) for x in ids]
    item_profiles = scipy.sparse.vstack(item_profiles_list)
    return item_profiles

def build_users_profile(person_id, interactions_indexed_df):
    interactions_person_df = interactions_indexed_df.loc[person_id]
    user_item_profiles = get_item_profiles(interactions_person_df['contentId'])
    
    user_item_strengths = np.array(interactions_person_df['eventStrength']).reshape(-1,1)
    #Weighted average of item profiles by the interactions strength
    user_item_strengths_weighted_avg = np.sum(user_item_profiles.multiply(user_item_strengths), axis=0) / np.sum(user_item_strengths)
    user_profile_norm = sklearn.preprocessing.normalize(user_item_strengths_weighted_avg)
    return user_profile_norm

def build_users_profiles(): 
    interactions_indexed_df = interactions_train_df[interactions_train_df['contentId'] \
                                                   .isin(article_df['contentId'])].set_index('personId')
    user_profiles = {}
    for person_id in interactions_indexed_df.index.unique():
        user_profiles[person_id] = build_users_profile(person_id, interactions_indexed_df)
    return user_profiles

In [53]:
user_profiles = build_users_profiles()
len(user_profiles)

1140

In [54]:
# show an example for user profiles, for user ID: -1479311724257856983
tfidf_feature_names = vectorizer.get_feature_names()
myprofile = user_profiles[-1479311724257856983]
print(myprofile.shape)
pd.DataFrame(sorted(zip(tfidf_feature_names, 
                        user_profiles[-1479311724257856983].flatten().tolist()), key=lambda x: -x[1])[:20],
             columns=['token', 'relevance'])

(1, 5000)


,token,relevance
0,learning,0.298732
1,machine learning,0.245992
2,machine,0.237843
3,google,0.202839
4,data,0.169776
5,ai,0.156203
6,algorithms,0.115666
7,like,0.097744
8,language,0.087609
9,people,0.082024


# Content-Based Filtering model

In [69]:
from sklearn.metrics.pairwise import cosine_similarity
class ContentBasedRecommender:
    
    MODEL_NAME = 'Content-Based'
    
    def __init__(self, items_df=None):
        self.item_ids = item_ids
        self.items_df = items_df
        
    def get_model_name(self):
        return self.MODEL_NAME
        
    def _get_similar_items_to_user_profile(self, person_id, topn=1000):
        #Computes the cosine similarity between the user profile and all item profiles
        cosine_similarities = cosine_similarity(user_profiles[person_id], tfidf_matrix)
        #Gets the top similar items
        similar_indices = cosine_similarities.argsort().flatten()[-topn:]
        #Sort the similar items by similarity
        similar_items = sorted([(item_ids[i], cosine_similarities[0,i]) for i in similar_indices], key=lambda x: -x[1])
        return similar_items
        
    def recommend_items(self, user_id, items_to_ignore=[], topn=10, verbose=False):
        similar_items = self._get_similar_items_to_user_profile(user_id)
        #Ignores items the user has already interacted
        similar_items_filtered = list(filter(lambda x: x[0] not in items_to_ignore, similar_items))
        
        recommendations_df = pd.DataFrame(similar_items_filtered, columns=['contentId', 'recStrength']) \
                                    .head(topn)

        if verbose:
            if self.items_df is None:
                raise Exception('"items_df" is required in verbose mode')

            recommendations_df = recommendations_df.merge(self.items_df, how = 'left', 
                                                          left_on = 'contentId', 
                                                          right_on = 'contentId')[['recStrength', 'contentId', 'title', 'url', 'lang']]


        return recommendations_df

In [70]:
content_based_recommender_model = ContentBasedRecommender(article_df)

# model evaluator

In [71]:
#Indexing by personId to speed up the searches during evaluation
interactions_full_indexed_df = interactions_full_df.set_index('personId')
interactions_train_indexed_df = interactions_train_df.set_index('personId')
interactions_test_indexed_df = interactions_test_df.set_index('personId')


In [72]:
import random
#Top-N accuracy metrics consts
EVAL_RANDOM_SAMPLE_NON_INTERACTED_ITEMS = 100

class ModelEvaluator:


    def get_not_interacted_items_sample(self, person_id, sample_size, seed=42):
        interacted_items = get_items_interacted(person_id, interactions_full_indexed_df)
        all_items = set(article_df['contentId'])
        non_interacted_items = all_items - interacted_items

        random.seed(seed)
        non_interacted_items_sample = random.sample(non_interacted_items, sample_size)
        return set(non_interacted_items_sample)

    def _verify_hit_top_n(self, item_id, recommended_items, topn):        
            try:
                index = next(i for i, c in enumerate(recommended_items) if c == item_id)
            except:
                index = -1
            hit = int(index in range(0, topn))
            return hit, index

    def evaluate_model_for_user(self, model, person_id):
        #Getting the items in test set
        interacted_values_testset = interactions_test_indexed_df.loc[person_id]
        if type(interacted_values_testset['contentId']) == pd.Series:
            person_interacted_items_testset = set(interacted_values_testset['contentId'])
        else:
            person_interacted_items_testset = set([int(interacted_values_testset['contentId'])])  
        interacted_items_count_testset = len(person_interacted_items_testset) 

        #Getting a ranked recommendation list from a model for a given user
        person_recs_df = model.recommend_items(person_id, 
                                               items_to_ignore=get_items_interacted(person_id, 
                                                                                    interactions_train_indexed_df), 
                                               topn=10000000000)

        hits_at_5_count = 0
        hits_at_10_count = 0
        #For each item the user has interacted in test set
        for item_id in person_interacted_items_testset:
            #Getting a random sample (100) items the user has not interacted 
            #(to represent items that are assumed to be no relevant to the user)
            non_interacted_items_sample = self.get_not_interacted_items_sample(person_id, 
                                                                          sample_size=EVAL_RANDOM_SAMPLE_NON_INTERACTED_ITEMS, 
                                                                          seed=item_id%(2**32))

            #Combining the current interacted item with the 100 random items
            items_to_filter_recs = non_interacted_items_sample.union(set([item_id]))

            #Filtering only recommendations that are either the interacted item or from a random sample of 100 non-interacted items
            valid_recs_df = person_recs_df[person_recs_df['contentId'].isin(items_to_filter_recs)]                    
            valid_recs = valid_recs_df['contentId'].values
            #Verifying if the current interacted item is among the Top-N recommended items
            hit_at_5, index_at_5 = self._verify_hit_top_n(item_id, valid_recs, 5)
            hits_at_5_count += hit_at_5
            hit_at_10, index_at_10 = self._verify_hit_top_n(item_id, valid_recs, 10)
            hits_at_10_count += hit_at_10

        #Recall is the rate of the interacted items that are ranked among the Top-N recommended items, 
        #when mixed with a set of non-relevant items
        recall_at_5 = hits_at_5_count / float(interacted_items_count_testset)
        recall_at_10 = hits_at_10_count / float(interacted_items_count_testset)

        person_metrics = {'hits@5_count':hits_at_5_count, 
                          'hits@10_count':hits_at_10_count, 
                          'interacted_count': interacted_items_count_testset,
                          'recall@5': recall_at_5,
                          'recall@10': recall_at_10}
        return person_metrics

    def evaluate_model(self, model):
        #print('Running evaluation for users')
        people_metrics = []
        for idx, person_id in enumerate(list(interactions_test_indexed_df.index.unique().values)):
            #if idx % 100 == 0 and idx > 0:
            #    print('%d users processed' % idx)
            person_metrics = self.evaluate_model_for_user(model, person_id)  
            person_metrics['_person_id'] = person_id
            people_metrics.append(person_metrics)
        print('%d users processed' % idx)

        detailed_results_df = pd.DataFrame(people_metrics) \
                            .sort_values('interacted_count', ascending=False)
        
        global_recall_at_5 = detailed_results_df['hits@5_count'].sum() / float(detailed_results_df['interacted_count'].sum())
        global_recall_at_10 = detailed_results_df['hits@10_count'].sum() / float(detailed_results_df['interacted_count'].sum())
        
        global_metrics = {'modelName': model.get_model_name(),
                          'recall@5': global_recall_at_5,
                          'recall@10': global_recall_at_10}    
        return global_metrics, detailed_results_df
    

In [73]:
def get_items_interacted(person_id, interactions_df):
    # Get the user's data and merge in the movie information.
    interacted_items = interactions_df.loc[person_id]['contentId']
    return set(interacted_items if type(interacted_items) == pd.Series else [interacted_items])

In [74]:

model_evaluator = ModelEvaluator()  
print('Evaluating Content-Based Filtering model...')
cb_global_metrics, cb_detailed_results_df = model_evaluator.evaluate_model(content_based_recommender_model)
print('\nGlobal metrics:\n%s' % cb_global_metrics)
cb_detailed_results_df.head(10)

Evaluating Content-Based Filtering model...
1139 users processed

Global metrics:
{'modelName': 'Content-Based', 'recall@5': 0.16287394528253643, 'recall@10': 0.2614420864229097}


,hits@5_count,hits@10_count,interacted_count,recall@5,recall@10,_person_id
76,15,24,192,0.078125,0.125000,3609194402293569455
17,18,29,134,0.134328,0.216418,-2626634673110551643
16,20,33,130,0.153846,0.253846,-1032019229384696495
10,32,47,117,0.273504,0.401709,-1443636648652872475
82,6,15,88,0.068182,0.170455,-2979881261169775358
161,11,23,80,0.137500,0.287500,-3596626804281480007
65,8,13,73,0.109589,0.178082,1116121227607581999
81,8,19,69,0.115942,0.275362,692689608292948411
106,3,9,69,0.043478,0.130435,-9016528795238256703
52,3,8,68,0.044118,0.117647,3636910968448833585


# store model artifact

In [77]:
import pickle
pickle.dump( item_ids, open( "../service/item_ids.p", "wb" ) )
pickle.dump( user_profiles, open( "../service/user_profiles.p", "wb" ) )
pickle.dump( tfidf_matrix, open( "../service/tfidf_matrix.p", "wb" ) )
pickle.dump( content_based_recommender_model, open( "../service/content_based_recommender_model.p", "wb" ) )
